In [2]:
!pip3 install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 4.5 MB/s eta 0:00:00 MB/s eta 0:00:01


In [1]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
import configparser as cp
#https://github.com/Refinitiv-API-Samples/Article.EikonAPI.Python.NewsSentimentAnalysis

In [2]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

In [3]:
from datetime import datetime, timedelta
end = datetime.today()
start = end - timedelta(days=(365.2*4))
print(start, end)

2019-04-18 22:19:07.091447 2023-04-18 17:31:07.091447


In [53]:
df = ek.get_news_headlines('R:MSFT.O AND Language:LEN', date_from=start, date_to=end, count=100)
df.head()

,versionCreated,text,storyId,sourceCode
2023-04-11 20:23:59.000,2023-04-11 20:23:59+00:00,UPDATE 1-Video gamers renew legal challenge to...,urn:newsml:newswire.refinitiv.com:20230411:nL1...,NS:RTRS
2023-04-11 20:05:02.003,2023-04-11 20:05:02.003000+00:00,Microsoft announces quarterly earnings release...,urn:newsml:newswire.refinitiv.com:20230411:nPn...,NS:PRN
2023-04-11 19:06:33.000,2023-04-11 19:06:33+00:00,"OpenAI to offer users up to $20,000 for report...",urn:newsml:newswire.refinitiv.com:20230411:nL4...,NS:RTRS
2023-04-11 18:36:13.000,2023-04-11 18:36:13+00:00,Sophos Group Ltd. - April showers Windows upda...,urn:newsml:newswire.refinitiv.com:20230411:nND...,NS:PUBT
2023-04-11 16:46:11.000,2023-04-11 16:46:11+00:00,UPDATE 1-Spain asks EU data protection board t...,urn:newsml:newswire.refinitiv.com:20230411:nL8...,NS:RTRS


In [54]:
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan

In [55]:
for idx, storyId in enumerate(df['storyId'].values):  #for each row in our df dataframe
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
        df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
        if sentA.sentiment.polarity >= 0.05: # attribute bucket to sentiment polartiy
            score = 'positive'
        elif  -.05 < sentA.sentiment.polarity < 0.05:
            score = 'neutral'
        else:
            score = 'negative'
        df['Score'].iloc[idx] = score #write score back to df
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score
2023-04-11 20:23:59.000,2023-04-11 20:23:59+00:00,UPDATE 1-Video gamers renew legal challenge to...,urn:newsml:newswire.refinitiv.com:20230411:nL1...,NS:RTRS,-0.031919,0.303232,neutral
2023-04-11 20:05:02.003,2023-04-11 20:05:02.003000+00:00,Microsoft announces quarterly earnings release...,urn:newsml:newswire.refinitiv.com:20230411:nPn...,NS:PRN,0.376420,0.493750,positive
2023-04-11 19:06:33.000,2023-04-11 19:06:33+00:00,"OpenAI to offer users up to $20,000 for report...",urn:newsml:newswire.refinitiv.com:20230411:nL4...,NS:RTRS,0.025749,0.520536,neutral
2023-04-11 18:36:13.000,2023-04-11 18:36:13+00:00,Sophos Group Ltd. - April showers Windows upda...,urn:newsml:newswire.refinitiv.com:20230411:nND...,NS:PUBT,0.101062,0.492039,positive
2023-04-11 16:46:11.000,2023-04-11 16:46:11+00:00,UPDATE 1-Spain asks EU data protection board t...,urn:newsml:newswire.refinitiv.com:20230411:nL8...,NS:RTRS,-0.010227,0.323864,neutral


In [56]:
start = df['versionCreated'].min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
end = df['versionCreated'].max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
Minute = ek.get_timeseries(["MSFT.O"], start_date=start, interval="minute")
Minute

MSFT.O,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2023-04-07 00:00:00,291.6,291.55,291.55,291.6,26,2262
2023-04-10 08:01:00,292.15,290.77,290.77,290.77,54,743
2023-04-10 08:02:00,291.49,291.02,291.02,291.06,25,168
2023-04-10 08:03:00,291.07,290.48,291.07,290.61,16,871
2023-04-10 08:04:00,290.98,290.55,290.62,290.98,4,60
...,...,...,...,...,...,...
2023-04-18 22:43:00,287.85,287.85,287.85,287.85,1,100
2023-04-18 22:44:00,287.85,287.85,287.85,287.85,2,17
2023-04-18 22:45:00,287.85,287.77,287.77,287.77,4,341


In [57]:
df['twoM'] = np.nan
df['fiveM'] = np.nan
df['tenM'] = np.nan
df['thirtyM'] = np.nan
df.head(2)

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-11 20:23:59.000,2023-04-11 20:23:59+00:00,UPDATE 1-Video gamers renew legal challenge to...,urn:newsml:newswire.refinitiv.com:20230411:nL1...,NS:RTRS,-0.031919,0.303232,neutral,NaN,NaN,NaN,NaN
2023-04-11 20:05:02.003,2023-04-11 20:05:02.003000+00:00,Microsoft announces quarterly earnings release...,urn:newsml:newswire.refinitiv.com:20230411:nPn...,NS:PRN,0.376420,0.493750,positive,NaN,NaN,NaN,NaN


In [58]:
df.tail()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-07 03:11:04.556,2023-04-07 03:11:04.556000+00:00,Maintenance And Support Of The Microsoft Envir...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
2023-04-07 03:06:46.411,2023-04-07 03:06:46.411000+00:00,Microsoft 365 Renewal Fy23,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA,0.375000,0.750000,positive,NaN,NaN,NaN,NaN
2023-04-07 03:03:37.641,2023-04-07 03:03:37.641000+00:00,Bid For - Procurement Of 40 Number Of Subscrip...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLTND,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
2023-04-07 01:41:27.360,2023-04-07 01:41:27.360000+00:00,USER REPORTS INDICATE MICROSOFT 365 IS HAVING ...,urn:newsml:newswire.refinitiv.com:20230407:nFW...,NS:RTRS,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
2023-04-06 23:10:58.597,2023-04-07 01:24:17.601000+00:00,Refinitiv Newscasts - Wall Street ends higher ...,urn:newsml:newswire.refinitiv.com:20230406:nRT...,NS:RTRS,0.091244,0.413056,positive,NaN,NaN,NaN,NaN


In [59]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    print(idx, newsDate)

0 2023-04-11T20:23:59.000000000
1 2023-04-11T20:05:02.003000000
2 2023-04-11T19:06:33.000000000
3 2023-04-11T18:36:13.000000000
4 2023-04-11T16:46:11.000000000
5 2023-04-11T16:41:30.000000000
6 2023-04-11T16:33:16.804000000
7 2023-04-11T16:00:00.000000000
8 2023-04-11T15:34:36.000000000
9 2023-04-11T14:49:54.000000000
10 2023-04-11T14:46:58.000000000
11 2023-04-11T14:37:45.219000000
12 2023-04-11T13:00:27.893000000
13 2023-04-11T13:00:01.861000000
14 2023-04-11T12:30:06.904000000
15 2023-04-11T12:30:03.491000000
16 2023-04-11T11:00:00.232000000
17 2023-04-11T09:01:19.371000000
18 2023-04-11T08:11:10.000000000
19 2023-04-11T03:23:52.874000000
20 2023-04-11T03:08:44.463000000
21 2023-04-11T02:16:34.705000000
22 2023-04-11T02:14:44.658000000
23 2023-04-10T23:22:29.915000000
24 2023-04-10T22:10:20.458000000
25 2023-04-10T20:44:42.993000000
26 2023-04-10T19:48:41.099000000
27 2023-04-10T18:00:22.000000000
28 2023-04-10T16:52:20.000000000
29 2023-04-10T16:01:08.000000000
30 2023-04-10T15:23:

In [60]:
idx = 0
newsDate = '2023-04-12T19:43:17.463000000'
sTime = df['versionCreated'][idx]
sTime = sTime.replace(second=0,microsecond=0)
sTime

Timestamp('2023-04-11 20:23:00+0000', tz='UTC')

In [61]:
t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
t0

283.3

In [62]:
Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=2))),3]

283.2

In [63]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    sTime = df['versionCreated'][idx]
    sTime = sTime.replace(second=0,microsecond=0)
    try:
        t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
        df['twoM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=2))),3]/(t0)-1)*100)
        df['fiveM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=5))),3]/(t0)-1)*100)
        df['tenM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=10))),3]/(t0)-1)*100) 
        df['thirtyM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=30))),3]/(t0)-1)*100)
    except:
        pass
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-11 20:23:59.000,2023-04-11 20:23:59+00:00,UPDATE 1-Video gamers renew legal challenge to...,urn:newsml:newswire.refinitiv.com:20230411:nL1...,NS:RTRS,-0.031919,0.303232,neutral,-0.035298,-0.028239,-0.035298,-0.035828
2023-04-11 20:05:02.003,2023-04-11 20:05:02.003000+00:00,Microsoft announces quarterly earnings release...,urn:newsml:newswire.refinitiv.com:20230411:nPn...,NS:PRN,0.376420,0.493750,positive,-0.010595,0.010595,0.000000,0.031785
2023-04-11 19:06:33.000,2023-04-11 19:06:33+00:00,"OpenAI to offer users up to $20,000 for report...",urn:newsml:newswire.refinitiv.com:20230411:nL4...,NS:RTRS,0.025749,0.520536,neutral,0.011977,-0.002114,-0.023250,-0.224041
2023-04-11 18:36:13.000,2023-04-11 18:36:13+00:00,Sophos Group Ltd. - April showers Windows upda...,urn:newsml:newswire.refinitiv.com:20230411:nND...,NS:PUBT,0.101062,0.492039,positive,-0.054774,-0.001761,0.022896,-0.019409
2023-04-11 16:46:11.000,2023-04-11 16:46:11+00:00,UPDATE 1-Spain asks EU data protection board t...,urn:newsml:newswire.refinitiv.com:20230411:nL8...,NS:RTRS,-0.010227,0.323864,neutral,-0.035287,-0.031758,0.064258,-0.215251


In [64]:
grouped = df.groupby(['Score']).mean()
grouped

,Polarity,Subjectivity,twoM,fiveM,tenM,thirtyM
Score,,,,,,
negative,-0.074614,0.318861,-0.042092,-0.040307,-0.066435,0.073719
neutral,0.003039,0.242634,0.018427,0.048040,0.042721,-0.026442
positive,0.140777,0.441769,0.013360,0.006920,0.016765,-0.018865
